In [2]:
import numpy as np

from analysis_tools.clustering import compute_partial_scores_matrix_fast
from analysis_tools.clustering import compute_clusters
from sklearn import cluster
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, fcluster,linkage
from scipy.cluster.hierarchy import set_link_color_palette


import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
from sklearn.manifold import MDS

from pyminimax import minimax

COMPUTATION OF SCORES MATRIX

Compute scores matrix (with Spectral descriptors OR Zernike descriptors)

In [ ]:
#Find example data of scores and entries in /pdb_shape_retrieval/example_data
#scores file should have three columns, first two colums #1 #2- entries IDs, third column #3 score
# entries_file - list of entry ids  

scores_file = './example_data/clustering_data/complexes/SARS_Spike_protein/zernike_scores.txt'
entries_file ='./example_data/clustering_data/complexes/SARS_Spike_protein/list_entries.txt'

#compute scores matrix
scores_matrix, labels = compute_partial_scores_matrix_fast(scores_file, entries_file)
scores_matrix_single=scores_matrix/scores_matrix.max()


#plot scores matrix
fig, (ax1) = plt.subplots(1, figsize=(4, 4))
sns.heatmap(scores_matrix_single,ax=ax1,vmin=0.0,cmap ='seismic')


Define a combined score using Zernike or Spectrals similarity scores to compute scores matrix  (optional)

CLUSTERING 

Define scores matrix to use for clustering

In [ ]:
#Define scores matrix to use: 

scores_matrix = scores_matrix_single

Hierarchical clustering

In [ ]:
#Define a threshold or number of clusters
threshold = None
n_clusters=2

#define clustering method ('ward' or 'average') and compute clusters
linkage_method="ward"

#Compute clusters
clusters,n_clusters,link_matrix,threshold_dist = compute_clusters(scores_matrix,labels,cluster,linkage_method, threshold, n_clusters)

print('optimal number of clusters',n_clusters)
print('threshold distance',threshold_dist)

#set clustering colour palette
set_link_color_palette(['red', 'blue', 'green', 'orange', 'purple','black'])

#plot dendogram
dendrogram(link_matrix,truncate_mode = "level", p=50,color_threshold=threshold_dist, labels=labels, above_threshold_color='lightgrey')
plt.axhline(threshold_dist, color='k', linestyle='--')
plt.xticks(rotation=90)  # Rotate labels vertically
plt.tight_layout() 
plt.show()